# Proyecto Final: Detección de SPAM mediante un clasificador bayesiano ingenuo.

In [1]:
from dataset import *
from naiveBayes import *

## Preparación de los datos.

In [2]:
# Leer el archivo csv.
sms_spam = pd.read_csv('datasets/SMSSpamCollection.csv', sep='\t', header=None, names=['Label', 'SMS'])

print("Tamaño del conjunto de datos:", sms_spam.shape)
sms_spam.head()

Tamaño del conjunto de datos: (5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Se cambia el orden del dataset de manera aleatoria.
sms_spam_reordenado = sms_spam.sample(frac = 1, random_state = 1)

# Se indica que el 80% de los datos se destinará para el entrenamiento.
indice_separacion_datos = round(len(sms_spam_reordenado) * 0.8)

# Separar el dataset en entrenamiento y prueba.
datos_entrenamiento = sms_spam_reordenado[:indice_separacion_datos].reset_index(drop = True)
datos_prueba = sms_spam_reordenado[indice_separacion_datos:].reset_index(drop = True)

print("Conjunto de entrenamiento:", datos_entrenamiento.shape)
print("Conjunto de prueba:", datos_prueba.shape)

Conjunto de entrenamiento: (4458, 2)
Conjunto de prueba: (1114, 2)


In [4]:
# Eliminar la puntuación y poner en minúsculas el texto.
datos_entrenamiento['SMS'] = datos_entrenamiento['SMS'].str.replace('\W', ' ', regex = True)
datos_entrenamiento['SMS'] = datos_entrenamiento['SMS'].str.lower()
datos_entrenamiento.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [5]:
# Encontrar todas las palabras presentes en los sms y crear un vocabulario.
datos_entrenamiento['SMS'] = datos_entrenamiento['SMS'].str.split()

vocabulario = []
for sms in datos_entrenamiento['SMS']:
   for palabra in sms:
      vocabulario.append(palabra)

vocabulario = list(set(vocabulario))

In [6]:
# Diccionario para contar las palabras en cada SMS.
palabras_por_sms = {palabra: [0] * len(datos_entrenamiento['SMS']) for palabra in vocabulario}

# Conteo de palabras en cada SMS.
for i, sms in enumerate(datos_entrenamiento['SMS']):
   for palabra in sms:
      palabras_por_sms[palabra][i] += 1

matriz_palabras = pd.DataFrame(palabras_por_sms)
matriz_palabras.head()

,sonyericsson,thurs,watchng,subscription,adult,treasure,txtauction,lotto,philosophical,world,...,juicy,dorothy,maximize,teletext,nat27081980,ofsi,missunderstding,jokin,decking,toxic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Agregar la etiqueta de clase al final del conjunto de datos.
datos_entrenamiento_limpios = pd.concat([matriz_palabras, datos_entrenamiento["Label"]], axis = 1)
datos_entrenamiento_limpios.head()

,sonyericsson,thurs,watchng,subscription,adult,treasure,txtauction,lotto,philosophical,world,...,dorothy,maximize,teletext,nat27081980,ofsi,missunderstding,jokin,decking,toxic,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham


In [8]:
# Convertir el nuevo dataset a csv.
datos_entrenamiento_limpios.to_csv("datasets/CleanSMSSpamCollection.csv", header = False, index = False)

In [9]:
# A partir del csv creado, instanciar un nuevo objeto de tipo 'DataSet'.
dataset = DataSet(nombre = "CleanSMSSpamCollection.csv")

In [10]:
# Se lleva a cabo el mismo tratamiento de datos para el conjunto de prueba, excepto que se deja como un DataFrame de pandas.
datos_prueba['SMS'] = datos_prueba['SMS'].str.replace('\W', ' ', regex = True)
datos_prueba['SMS'] = datos_prueba['SMS'].str.lower()

datos_prueba['SMS'] = datos_prueba['SMS'].str.split()

palabras_por_sms_prueba = {palabra: [0] * len(datos_prueba['SMS']) for palabra in vocabulario}

for i, sms in enumerate(datos_prueba['SMS']):
   for palabra in sms:
      if palabra in vocabulario:
         palabras_por_sms_prueba[palabra][i] += 1

matriz_palabras_prueba = pd.DataFrame(palabras_por_sms_prueba)
datos_prueba_limpios = pd.concat([matriz_palabras_prueba, datos_prueba["Label"]], axis = 1)
datos_prueba_limpios.head()

,sonyericsson,thurs,watchng,subscription,adult,treasure,txtauction,lotto,philosophical,world,...,dorothy,maximize,teletext,nat27081980,ofsi,missunderstding,jokin,decking,toxic,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spam
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham


## Entrenamiento del modelo.

In [11]:
# Se instancia y se entrena un clasificador bayesiano ingenuo discreto.
naive_bayes_discreto = Clasificador_Naive_Bayes_Discreto(dataset)
naive_bayes_discreto.entrenar()

In [12]:
# Se instancia y se entrena un clasificador bayesiano ingenuo continuo.
naive_bayes_continuo = Clasificador_Naive_Bayes_Continuo(dataset)
naive_bayes_continuo.entrenar()

## Evaluación del modelo.

In [13]:
# Función para generar la matriz de confusión.
def matriz_de_confusion(lista, clases):
      columna_0 =['', 'Valor', 'Verdadero']
      columna_1 =['-', clases[0], clases[1]]
      columna_2 =[clases[0], lista[0], lista[2]]
      columna_3 =[clases[1], lista[1], lista[3]]
      tabla = zip(columna_0, columna_1, columna_2, columna_3)
      encabezado = ['', '', 'Valor', 'Predicho']  
      return print(tabulate(tabla, headers = encabezado, floatfmt = ".4f")) 

In [14]:
# Se instancian variables para calcular las métricas.
tp, tn, fp, fn = 0, 0, 0, 0

# Evaluación del modelo para el caso discreto.
for i in datos_prueba_limpios.iterrows():
    fila = list(i[1])
    clase = fila[-1]
    prediccion = naive_bayes_discreto.predecir(fila[:-1])

    if (clase == "spam" and prediccion == "ham"):
        fp += 1
    if (clase == "ham" and prediccion == "ham"):
        tp += 1
    if (clase == "spam" and prediccion == "spam"):
        tn += 1
    if (clase == "ham" and prediccion == "spam"):
        fn += 1

In [15]:
# Matriz de confusión para el caso discreto.
matriz_de_confusion([tp, fn, fp, tn], ['ham', 'spam'])

                 Valor    Predicho
---------  ----  -------  ----------
           -     ham      spam
Valor      ham   966      1
Verdadero  spam  35       112


In [16]:
# Métricas de desempeño para el caso discreto.
acc = (tp+tn)/(tp+tn+fp+fn)
pre = (tp)/(tp+fp)
rec = (tp)/(tp+fn)
f1 = 2 * (pre*rec)/(pre+rec)
spe = (tn)/(tn+fp)

print('Accuracy: {}'.format(acc))
print('Precision: {}'.format(pre))
print('Recall: {}'.format(rec))
print('F1: {}'.format(f1))
print('Specificity: {}'.format(f1))

Accuracy: 0.9676840215439856
Precision: 0.965034965034965
Recall: 0.9989658738366081
F1: 0.9817073170731707
Specificity: 0.9817073170731707


In [17]:
# Se instancian variables para calcular las métricas.
tp, tn, fp, fn = 0, 0, 0, 0

# Evaluación del modelo para el caso continuo.
for i in datos_prueba_limpios.iterrows():
    fila = list(i[1])
    clase = fila[-1]
    prediccion = naive_bayes_continuo.predecir(fila[:-1])

    if (clase == "spam" and prediccion == "ham"):
        fp += 1
    if (clase == "ham" and prediccion == "ham"):
        tp += 1
    if (clase == "spam" and prediccion == "spam"):
        tn += 1
    if (clase == "ham" and prediccion == "spam"):
        fn += 1

In [18]:
# Matriz de confusión para el caso continuo.
matriz_de_confusion([tp, fn, fp, tn], ['ham', 'spam'])

                 Valor    Predicho
---------  ----  -------  ----------
           -     ham      spam
Valor      ham   926      41
Verdadero  spam  130      17


In [19]:
# Métricas de desempeño para el caso continuo.
acc = (tp+tn)/(tp+tn+fp+fn)
pre = (tp)/(tp+fp)
rec = (tp)/(tp+fn)
f1 = 2 * (pre*rec)/(pre+rec)
spe = (tn)/(tn+fp)

print('Accuracy: {}'.format(acc))
print('Precision: {}'.format(pre))
print('Recall: {}'.format(rec))
print('F1: {}'.format(f1))
print('Specificity: {}'.format(f1))

Accuracy: 0.8464991023339318
Precision: 0.8768939393939394
Recall: 0.9576008273009308
F1: 0.9154720711814138
Specificity: 0.9154720711814138


## Aplicación del modelo.

### Separación de archivos en carpetas.

In [20]:
'''
    Leer los mensajes contenidos en archivos .sms en la carpeta tests/unsorted.
'''

archivos_df = pd.DataFrame() # Dataframe para guardar los datos de los SMS.
nombres_archivos = pd.DataFrame() # Dataframe para guardar los nombres de archivos asociados a cada SMS.

directorio = "tests/unsorted" # Directorio base de SMS no clasificados.

idx = 0 # Índice de lectura de SMS.

for nombre_archivo in os.listdir(directorio): # Iterar a través de los archivos del directorio.
    ruta = os.path.join(directorio, nombre_archivo) # Determinar la ruta completa del archivo.
    if os.path.isfile(ruta): # Verfificar si la ruta es un archivo.
        archivo = open(ruta, "r") # Abrir el archivo.
        
        entrada_archivo = pd.DataFrame({"Label": nombre_archivo.split("_")[1].split(".")[0], "SMS": archivo.read()}, index = [idx]) # Crear la entrada del archhivo leído.
        archivos_df = pd.concat([archivos_df, entrada_archivo]) # Agregar la entrada al dataframe.
        
        nombres_archivos = pd.concat([nombres_archivos, pd.DataFrame({"filename": ruta}, index = [idx])]) # Agregar la entrada con la ruta del archivo.
        idx += 1 # Aumentar el índice.
        archivo.close() # Cerrar el archivo.        

In [21]:
# Tratamiento de datos para los mensajes leídos.

archivos_df['SMS'] = archivos_df['SMS'].str.replace('\W', ' ', regex = True)
archivos_df['SMS'] = archivos_df['SMS'].str.lower()

archivos_df['SMS'] = archivos_df['SMS'].str.split()

palabras_por_sms_archivos = {palabra: [0] * len(archivos_df['SMS']) for palabra in vocabulario}

for i, sms in enumerate(archivos_df['SMS']):
   for palabra in sms:
      if palabra in vocabulario:
         palabras_por_sms_archivos[palabra][i] += 1

matriz_palabras_archivos = pd.DataFrame(palabras_por_sms_archivos)
datos_archivos_limpios = pd.concat([matriz_palabras_archivos, archivos_df["Label"]], axis = 1)
datos_archivos_limpios.head()

,sonyericsson,thurs,watchng,subscription,adult,treasure,txtauction,lotto,philosophical,world,...,dorothy,maximize,teletext,nat27081980,ofsi,missunderstding,jokin,decking,toxic,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham


In [22]:
# Aplicación del modelo para el caso discreto.
for i in datos_archivos_limpios.iterrows():
    fila = list(i[1])
    clase = fila[-1]
    prediccion = naive_bayes_discreto.predecir(fila[:-1])
    
    entrada_recuperada = nombres_archivos.iloc[i[0]]  # Recuperar entrada del dataframe de rutas de archivo 
    ruta_completa = entrada_recuperada['filename']    # Recuperar la ruta del archivo
    nombre_recuperado = ruta_completa.split("\\")[1]  # Recuperar el nombre del archivo: puede que deba cambiarlo a ruta_completa.split("/")[2]
    
    '''
        En este condicional se verifica el tipo de clasificación determinada
        y se realizan los siguientes procesos:
            1. Determinar la ruta de destino
            2. Crear el directorio de destino si no existe
            3. Copiar el archivo original a la ruta de destino.
    '''

    if prediccion == "spam":
        ruta_destino = f"tests/sorted/discrete/spam/{nombre_recuperado}"
        os.makedirs(os.path.dirname(ruta_destino), exist_ok = True)
        shutil.copy(ruta_completa, ruta_destino)
    else:
        ruta_destino = f"tests/sorted/discrete/ham/{nombre_recuperado}"
        os.makedirs(os.path.dirname(ruta_destino), exist_ok = True)
        shutil.copy(ruta_completa, ruta_destino)      

In [23]:
# Aplicación del modelo para el caso continuo.
for i in datos_archivos_limpios.iterrows():
    fila = list(i[1])
    clase = fila[-1]
    prediccion = naive_bayes_continuo.predecir(fila[:-1])
    
    entrada_recuperada = nombres_archivos.iloc[i[0]]  # Recuperar entrada del dataframe de rutas de archivo 
    ruta_completa = entrada_recuperada['filename']    # Recuperar la ruta del archivo
    nombre_recuperado = ruta_completa.split("\\")[1]  # Recuperar el nombre del archivo: puede que deba cambiarlo a ruta_completa.split("/")[2]
    
    '''
        En este condicional se verifica el tipo de clasificación determinada
        y se realizan los siguientes procesos:
            1. Determinar la ruta de destino
            2. Crear el directorio de destino si no existe
            3. Copiar el archivo original a la ruta de destino.
    '''

    if prediccion == "spam":
        ruta_destino = f"tests/sorted/continuous/spam/{nombre_recuperado}"
        os.makedirs(os.path.dirname(ruta_destino), exist_ok = True)
        shutil.copy(ruta_completa, ruta_destino)
    else:
        ruta_destino = f"tests/sorted/continuous/ham/{nombre_recuperado}"
        os.makedirs(os.path.dirname(ruta_destino), exist_ok = True)
        shutil.copy(ruta_completa, ruta_destino)      

### Mensaje personalizado.

In [28]:
# Se ingresa el mensaje a detectar como 'spam' o 'ham'.
mensaje_de_texto = input()
print(mensaje_de_texto)

You have been chosen to receive a one-time $500 shopping voucher! Click HERE to redeem your offer now.


In [29]:
datos_mensaje = pd.DataFrame({"Label": "spam", "SMS": mensaje_de_texto}, index = [0]) # Crear el DataFrame.

# Tratamiento de datos para el mensaje ingresado.
datos_mensaje['SMS'] = datos_mensaje['SMS'].str.replace('\W', ' ', regex = True)
datos_mensaje['SMS'] = datos_mensaje['SMS'].str.lower()
datos_mensaje['SMS'] = datos_mensaje['SMS'].str.split()

palabras_por_mensaje_sms = {palabra: [0] * len(datos_mensaje['SMS']) for palabra in vocabulario}

for i, sms in enumerate(datos_mensaje['SMS']):
   for palabra in sms:
      if palabra in vocabulario:
         palabras_por_mensaje_sms[palabra][i] += 1
         
matriz_palabras_mensaje = pd.DataFrame(palabras_por_mensaje_sms)
matriz_palabras_mensaje.head()

,sonyericsson,thurs,watchng,subscription,adult,treasure,txtauction,lotto,philosophical,world,...,juicy,dorothy,maximize,teletext,nat27081980,ofsi,missunderstding,jokin,decking,toxic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Aplicación del modelo para el caso discreto.
resultado_discreto = naive_bayes_discreto.predecir(list(matriz_palabras_mensaje.iloc[0]))
print("Este mensaje fue detectado como", resultado_discreto)

Este mensaje fue detectado como spam


In [31]:
# Aplicación del modelo para el caso continuo.
resultado_continuo = naive_bayes_continuo.predecir(list(matriz_palabras_mensaje.iloc[0]))
print("Este mensaje fue detectado como", resultado_continuo)

Este mensaje fue detectado como ham
